In [3]:
import sys
import pathlib
src_path = pathlib.Path().absolute().parent / "src"
sys.path.append(str(src_path))
import data_functions
import custom_transformers as ct
import pandas as pd
import numpy as np

import pickle

from sklearn.compose import  make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

In [4]:
x_train, x_test, y_train = data_functions.get_dataframes()

In [18]:
# simple_impute_strict = make_column_transformer((SimpleImputer(), data_functions.get_numeric_features(data_functions.get_strict_features())), remainder = 'passthrough')
# ss_strict = make_column_transformer((StandardScaler(), data_functions.get_numeric_features(data_functions.get_strict_features())), remainder = 'passthrough')
# ohe_strict = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore', sparse = False), data_functions.get_categorical_features(data_functions.get_strict_features())), remainder = 'passthrough')

# simple_impute_cy = make_column_transformer((SimpleImputer(missing_values = 0.0), ['construction_year']), remainder = 'passthrough')
# transform_construction = make_column_transformer((ct.TransformConstructionYear(), ['construction_year']), remainder = 'passthrough')

# simple_impute_loose = make_column_transformer((SimpleImputer(), data_functions.get_numeric_features(data_functions.get_loose_features())), remainder = 'passthrough')
# ss_loose = make_column_transformer((StandardScaler(), data_functions.get_numeric_features(data_functions.get_loose_features())), remainder = 'passthrough')
# ohe_loose = make_column_transformer((OneHotEncoder(catgories = 'auto', handle_unknown = 'ignore', sparse = False), data_functions.get_categorical_features(data_functions.get_loose_features())), remainder = 'passthrough')

# strict_preprocessing = make_pipeline(ct.ChooseStrictFeatures(),
#                                     ct.BinInstaller(),
#                                     simple_impute_cy,
#                                     simple_impute_strict,
#                                     transform_construction,
#                                     ss_strict,
#                                     ohe_strict)
                                    

# loose_preprocessing = make_pipeline(ct.ChooseLooseFeatures(),
#                                     ct.BinInstaller(),
#                                     simple_impute_cy,
#                                     transform_construction,
#                                     simple_impute_loose,
#                                     ss_loose,
#                                     ohe_loose)

In [26]:
# strict_preprocessing = make_pipeline(ct.ChooseStrictFeatures(),
#                                     ct.BinInstaller(),
#                                     make_column_transformer((SimpleImputer(missing_values = 0.0), ['construction_year']),
#                                                             (SimpleImputer(), data_functions.get_numeric_features(data_functions.get_strict_features())),
#                                                             (ct.TransformConstructionYear(), ['construction_year']),
#                                                             (StandardScaler(), data_functions.get_numeric_features(data_functions.get_strict_features())),
#                                                             (OneHotEncoder(categories = 'auto', handle_unknown = 'ignore', sparse = False), data_functions.get_categorical_features(data_functions.get_strict_features())),
#                                                             remainder = 'drop'))
                                    

# loose_preprocessing = make_pipeline(ct.ChooseLooseFeatures(),
#                                     ct.BinInstaller(),
#                                     make_column_transformer((SimpleImputer(missing_values = 0.0), ['construction_year']),
#                                                             (SimpleImputer(), data_functions.get_numeric_features(data_functions.get_loose_features())),
#                                                             (ct.TransformConstructionYear(), ['construction_year']),
#                                                             (StandardScaler(), data_functions.get_numeric_features(data_functions.get_loose_features())),
#                                                             (OneHotEncoder(categories = 'auto', handle_unknown = 'ignore', sparse = False), data_functions.get_categorical_features(data_functions.get_loose_features())),
#                                                             remainder = 'drop'))

In [49]:
strict_pp_pipe = Pipeline([('feature_selector', ct.ChooseStrictFeatures()),
                           ('bin_installer', ct.BinInstaller()),
                           ('column_transformer', make_column_transformer((SimpleImputer(missing_values = 0.0), ['construction_year']),
                                                            (SimpleImputer(), data_functions.get_numeric_features(data_functions.get_strict_features())),
                                                            (ct.TransformConstructionYear(), ['construction_year']),
                                                            (StandardScaler(), data_functions.get_numeric_features(data_functions.get_strict_features())),
                                                            (OneHotEncoder(handle_unknown = 'ignore', sparse = False), data_functions.get_categorical_features(data_functions.get_strict_features())),
                                                            remainder = 'drop'))])
strict_resamp_pipe = Pipeline([('pp_pipeline', strict_pp_pipe), ('up_sample', SMOTE(random_state = 42))])

TypeError: All intermediate steps of the chain should be estimators that implement fit and transform or fit_resample. 'Pipeline(steps=[('feature_selector', ChooseStrictFeatures()),
                ('bin_installer', BinInstaller()),
                ('column_transformer',
                 ColumnTransformer(transformers=[('simpleimputer-1',
                                                  SimpleImputer(missing_values=0.0),
                                                  ['construction_year']),
                                                 ('simpleimputer-2',
                                                  SimpleImputer(),
                                                  ['amount_tsh', 'gps_height',
                                                   'population',
                                                   'construction_year']),
                                                 ('transformconstructionyear',
                                                  T...rmConstruction

In [36]:
resampled_data = strict_pp_pipe.fit_resample(x_train, y_train)

In [45]:
x_tr_pp = pd.DataFrame(resampled_data[0])

In [46]:
x_tr_pp

,0,1,2,3,4,5,6,7,8,9,...,625,626,627,628,629,630,631,632,633,634
0,1999.000000,6000.0,1390.000000,109.000000,1999.000000,21.000000,1.895665,1.041252,-0.150399,0.733857,...,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000
1,2010.000000,0.0,1399.000000,280.000000,2010.000000,10.000000,-0.105970,1.054237,0.212290,0.745416,...,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000
2,2009.000000,25.0,686.000000,250.000000,2009.000000,11.000000,-0.097630,0.025541,0.148660,0.744365,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.000000
3,1986.000000,0.0,263.000000,58.000000,1986.000000,34.000000,-0.105970,-0.584751,-0.258570,0.720196,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.000000
4,1996.814686,0.0,0.000000,0.000000,0.000000,0.000000,-0.105970,-0.964200,-0.381587,-1.366788,...,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96772,1996.814686,0.0,0.000000,0.000000,0.000000,0.000000,-0.105970,-0.964200,-0.381587,-1.366788,...,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000
96773,1994.572299,0.0,336.283104,1.000000,1994.572299,25.427701,-0.105970,-0.479020,-0.379466,0.729204,...,0.427701,0.000000,0.572299,0.0,0.427701,0.572299,0.0,0.000000,0.0,0.000000
96774,1977.193603,0.0,82.701599,1.000000,1977.193603,42.806397,-0.105970,-0.844881,-0.379466,0.710942,...,0.000000,0.298401,0.701599,0.0,0.701599,0.000000,0.0,0.000000,0.0,0.298401
96775,1988.000000,0.0,-35.088579,396.519036,1988.000000,32.000000,-0.105970,-1.014825,0.459425,0.722298,...,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.565398,0.0,0.434602


In [47]:
x_tr_pp = x_tr_pp.drop([0, 1, 2, 3, 4, 5], axis = 1)

In [48]:
x_tr_pp

,6,7,8,9,10,11,12,13,14,15,...,625,626,627,628,629,630,631,632,633,634
0,1.895665,1.041252,-0.150399,0.733857,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000
1,-0.105970,1.054237,0.212290,0.745416,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000
2,-0.097630,0.025541,0.148660,0.744365,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.000000
3,-0.105970,-0.584751,-0.258570,0.720196,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.0,0.000000,0.0,0.000000
4,-0.105970,-0.964200,-0.381587,-1.366788,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96772,-0.105970,-0.964200,-0.381587,-1.366788,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,1.000000,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.000000
96773,-0.105970,-0.479020,-0.379466,0.729204,0.0,0.0,0.0,0.0,0.0,0.0,...,0.427701,0.000000,0.572299,0.0,0.427701,0.572299,0.0,0.000000,0.0,0.000000
96774,-0.105970,-0.844881,-0.379466,0.710942,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.298401,0.701599,0.0,0.701599,0.000000,0.0,0.000000,0.0,0.298401
96775,-0.105970,-1.014825,0.459425,0.722298,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.0,0.565398,0.0,0.434602


In [ ]:
strict_model = Pipeline([('preprocessor', strict_pp_pipe), ('dTree_classifier', DecisionTreeClassifier())])

In [28]:
loose_model = make_pipeline(loose_preprocessing, DecisionTreeClassifier())

In [29]:
cross_val_score(strict_model, x_train, y_train, cv = 5)

array([0.75465028, 0.7538086 , 0.76010101, 0.75151515, 0.74793736])

In [30]:
cross_val_score(loose_model, x_train, y_train, cv = 5)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:530: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Input contains NaN

In [41]:
strict_pickle_path = src_path / 'strict_pre_pipeline_v4.pkl'
#loose_pickle_path = src_path / 'loose_pre_pipeline_v2.pkl'
file = open(strict_pickle_path, 'wb')
pickle.dump(strict_pp_pipe, file)
file.close()
# file = open(loose_pickle_path, 'wb')
# pickle.dump(loose_preprocessing, file)
# file.close()

0       20709
2010     2645
2008     2613
2009     2533
2000     2091
2007     1587
2006     1471
2003     1286
2011     1256
2004     1123
2012     1084
2002     1075
1978     1037
1995     1014
2005     1011
1999      979
1998      966
1990      954
1985      945
1980      811
1996      811
1984      779
1982      744
1994      738
1972      708
1974      676
1997      644
1992      640
1993      608
2001      540
1988      521
1983      488
1975      437
1986      434
1976      414
1970      411
1991      324
1989      316
1987      302
1981      238
1977      202
1979      192
1973      184
2013      176
1971      145
1960      102
1967       88
1963       85
1968       77
1969       59
1964       40
1962       30
1961       21
1965       19
1966       17
Name: construction_year, dtype: int64